# Template Matching OCR

In [4]:
import cv2
import numpy as np
import os

def load_image(image_path):
    """Load the image and convert to grayscale"""
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not load image from {image_path}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray, img

def load_templates(template_dir):
    """Load all template images from a directory"""
    templates = {}
    
    if not os.path.exists(template_dir):
        raise ValueError(f"Template directory {template_dir} does not exist!")
    
    template_files = [f for f in os.listdir(template_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    if not template_files:
        raise ValueError(f"No template images found in {template_dir}")
    
    for file in template_files:
        # Extract character name from filename (assuming format like "A.png", "B.png", etc.)
        char_name = os.path.splitext(file)[0]
        template_path = os.path.join(template_dir, file)
        template = cv2.imread(template_path, 0)  # Read as grayscale
        
        if template is not None:
            templates[char_name] = template
            print(f"Loaded template: {char_name}")
        else:
            print(f"Warning: Could not load template {template_path}")
    
    return templates

def match_character(roi, templates, threshold=0.7):
    """Match a character region against all templates"""
    best_match = None
    best_score = -1
    
    for char, template in templates.items():
        # Resize template to match ROI size if needed
        if roi.shape != template.shape:
            resized_template = cv2.resize(template, (roi.shape[1], roi.shape[0]))
        else:
            resized_template = template
            
        result = cv2.matchTemplate(roi, resized_template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(result)
        
        if max_val > best_score and max_val > threshold:
            best_score = max_val
            best_match = char
    
    return best_match if best_score > threshold else "?"

def extract_text_with_templates(image, x, y, w, h, templates):
    """Extract text from a region using template matching"""
    roi = image[y:y+h, x:x+w]
    
    # Preprocess the ROI
    _, binary = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    
    text = ""
    for contour in contours:
        x_c, y_c, w_c, h_c = cv2.boundingRect(contour)
        
        if w_c < 5 or h_c < 10:
            continue
            
        char_roi = roi[y_c:y_c+h_c, x_c:x_c+w_c]
        
        char = match_character(char_roi, templates)
        text += char
    
    return text

def find_field_locations(image, field_templates):
    """Find locations of specific fields using their label templates"""
    field_regions = {}
    
    for field_name, template in field_templates.items():
        # Perform template matching
        result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        if max_val > 0.6:  # Good match threshold
            # Get template dimensions
            h, w = template.shape
            
            # Estimate position of the field value (assuming it's to the right of the label)
            field_x = max_loc[0] + w + 10  # 10 pixels padding
            field_y = max_loc[1]
            field_width = 200  # Estimated width of the field value
            field_height = h
            
            field_regions[field_name] = (field_x, field_y, field_width, field_height)
            
            # For debugging: visualize the found template
            debug_img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            cv2.rectangle(debug_img, max_loc, (max_loc[0] + w, max_loc[1] + h), (0, 255, 0), 2)
            cv2.putText(debug_img, field_name, (max_loc[0], max_loc[1]-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imshow(f"Found {field_name} template", debug_img)
            cv2.waitKey(0)
    
    return field_regions

def main():
    # Path to your image and templates
    image_path = "id"
    template_dir = "templates"  # Directory with all templates
    
    # Load image
    gray, original = load_image(image_path)
    
    # Load all templates from the directory
    try:
        templates = load_templates(template_dir)
        print(f"Loaded {len(templates)} templates")
    except ValueError as e:
        print(e)
        return
    
    # Separate character templates from field templates
    char_templates = {}
    field_templates = {}
    
    for name, template in templates.items():
        # Assuming field templates have longer names (like "STUDENT", "ID")
        # and character templates are single characters
        if len(name) == 1:
            char_templates[name] = template
        else:
            field_templates[name] = template
    
    print(f"Character templates: {len(char_templates)}")
    print(f"Field templates: {len(field_templates)}")
    
    # Find field locations if we have field templates
    if field_templates:
        field_regions = find_field_locations(gray, field_templates)
        
        # Extract text from each field
        for field_name, region in field_regions.items():
            text = extract_text_with_templates(gray, *region, char_templates)
            print(f"{field_name}: {text}")
    else:
        # Manual region definitions (fallback if no field templates)
        print("No field templates found. Using manual region definitions.")
        
        # Define regions manually (you'll need to adjust these)
        student_region = (150, 100, 280, 40)  # (x, y, w, h)
        id_region = (150, 135, 280, 40)
        
        # Extract text from regions
        student_name = extract_text_with_templates(gray, *student_region, char_templates)
        student_id = extract_text_with_templates(gray, *id_region, char_templates)
        
        print(f"Student Name: {student_name}")
        print(f"Student ID: {student_id}")
        
        # Visualize the regions (for debugging)
        debug_img = original.copy()
        cv2.rectangle(debug_img, 
                     (student_region[0], student_region[1]), 
                     (student_region[0] + student_region[2], student_region[1] + student_region[3]), 
                     (0, 255, 0), 2)
        cv2.rectangle(debug_img, 
                     (id_region[0], id_region[1]), 
                     (id_region[0] + id_region[2], id_region[1] + id_region[3]), 
                     (255, 0, 0), 2)
        
        # Add labels to original fields
        cv2.putText(debug_img, "Name", (student_region[0], student_region[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(debug_img, "ID", (id_region[0], id_region[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # ✅ Display name and ID on top-left
        cv2.putText(debug_img, f"Name: {student_name}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(debug_img, f"ID: {student_id}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Show image
        cv2.imshow("Template Matching OCR Result", debug_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded template: 0
Loaded template: 1
Loaded template: 2
Loaded template: 3
Loaded template: 4
Loaded template: 5
Loaded template: 6
Loaded template: 7
Loaded template: 8
Loaded template: 9
Loaded template: A
Loaded template: B
Loaded template: C
Loaded template: D
Loaded template: E
Loaded template: F
Loaded template: G
Loaded template: H
Loaded template: I
Loaded template: J
Loaded template: K
Loaded template: L
Loaded template: M
Loaded template: N
Loaded template: O
Loaded template: P
Loaded template: Q
Loaded template: R
Loaded template: S
Loaded template: T
Loaded template: U
Loaded template: V
Loaded template: W
Loaded template: X
Loaded template: Y
Loaded template: Z
Loaded 36 templates
Character templates: 36
Field templates: 0
No field templates found. Using manual region definitions.
Student Name: ?????W???
Student ID: ??W?????


# Tesseract OCR

In [ ]:
import cv2 as cv
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

img = cv.imread("id")
if img is None:
    raise ValueError("无法读取 id.png，请确认路径正确")

student_region = (150, 100, 280, 40)  # (x, y, w, h)
id_region = (150, 135, 280, 40)

def preprocess_and_ocr(img, region):
    x, y, w, h = region
    cropped = img[y:y+h, x:x+w]
    gray = cv.cvtColor(cropped, cv.COLOR_BGR2GRAY)
    _, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    text = pytesseract.image_to_string(binary, lang='chi_sim+eng', config='--psm 7').strip()
    return text

student_name = preprocess_and_ocr(img, student_region)
student_id = preprocess_and_ocr(img, id_region)

cv.rectangle(img, (student_region[0], student_region[1]), (student_region[0]+student_region[2], student_region[1]+student_region[3]), (0,255,0), 2)
cv.rectangle(img, (id_region[0], id_region[1]), (id_region[0]+id_region[2], id_region[1]+id_region[3]), (255,0,0), 2)

cv.putText(img, f"Name: {student_name}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
cv.putText(img, f"ID: {student_id}", (10, 60), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)

cv.imshow("Tesseract OCR Result", img)
cv.waitKey(0)
cv.destroyAllWindows()